In [ ]:
import torch
import torch.nn as nn
from torch.nn import ReLU
from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.integrate as integrate
import scipy.special as special

In [ ]:
#radius of the ball
radius = 2

#parameters of the OU process
theta = 1
sigma = 0.5
lambda_ = theta/sigma**2

#definition of the true solution for comparison

def true_solution(x, y, theta=theta, lambda_=lambda_):
    rho = np.sqrt(x**2 + y**2)
    integral, _ = integrate.quad(lambda t : np.exp(lambda_ * t**2) * special.gammainc(1, lambda_ * t**2) / t, rho, radius)
    return integral/theta

#make the true_solution compatible with numpy arrays
true_solution_vectorized = np.vectorize(true_solution)

#plotting the function (inspiration taken from https://stackoverflow.com/questions/27606079)
n_points = 400
tt = np.linspace(-1, 1, n_points) * radius
xx, yy = np.meshgrid(tt, tt)  # create unit square grid

xx, yy = np.where(xx**2 + yy**2 <= radius**2, xx, 0), np.where(xx**2 + yy**2 <= radius**2 , yy, 0) #(https://stackoverflow.com/questions/15733530/)
zz = true_solution_vectorized(xx,yy)

In [ ]:
#circle
a = np.linspace(0, 2*np.pi, 50)
cx,cy = np.cos(a) * radius , np.sin(a)*radius

fig, ax = plt.subplots()
ax.plot(cx, cy,'k-', alpha=.2)

contour = ax.contourf(xx, yy, zz, levels=50)  # 50 contour levels
cb = fig.colorbar(contour, ax=ax)

As we can see, the true solution blows up extremely fast away from the boundary, and is thus likely to be hard to learn.

In [ ]:
#define ReLU^k activation

class RePU(nn.Module):

    def __init__(self, power = 2):
        super(RePU, self).__init__()
        self.power = power

    def forward(self, x):
        return torch.pow(torch.relu(x), self.power)


In [ ]:
iter = 0
width = 1000
losses = []

class StandardPINN(nn.Module):

    def __init__(self, power = 2, width = width):
        super().__init__()

        self.layers = torch.nn.Sequential(torch.nn.Linear(2, width),
                                          RePU(power),
                                          torch.nn.Linear(width, 1))

    def forward(self, x):
        return self.layers(x)

def derivative(dy: torch.Tensor, x: torch.Tensor, order: int = 1) -> torch.Tensor:
    """
    This function calculates the derivative of the model at x_f
    """
    for i in range(order):
        dy = torch.autograd.grad(
            dy, x, grad_outputs = torch.ones_like(dy), create_graph=True, retain_graph=True
        )[0]
    return dy

def u_function(model: StandardPINN, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """
    This function evaluates the model on the input x
    """
    model_input = torch.stack((x, y), axis = 1)
    return model(model_input)


def residual(model, x_f, y_f):
    u = u_function(model, x_f, y_f)
    u_x = derivative(u, x_f, order=1)
    u_y = derivative(u, y_f, order=1)
    u_xx = derivative(u, x_f, order=2)
    u_yy = derivative(u, y_f, order=2)
    res = - theta * (x_f * u_x + y_f * u_y) \
        + sigma**2 * (u_xx + u_yy)/2 \
        + 1
    return res

def loss_function(model: StandardPINN, x_c: torch.Tensor, y_c: torch.Tensor, x_b: torch.Tensor, y_b: torch.Tensor) -> torch.Tensor:
    """
    This function evaluates the physics governing the model on the input x_f
    """
    domain_residual = residual(model, x_c, y_c).pow(2).mean() #residual
    boundary_residual = (u_function(model, x_b, y_b)).pow(2).mean() #boundary
    return domain_residual + boundary_residual

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.01)


def closure(model: StandardPINN, optimizer, X_c_train: torch.Tensor, X_b_train:torch.Tensor) -> torch.Tensor:
    """
    In order to use the LBFGS optimizer, we need to define a closure function. This function is called by the optimizer
    and the optimizer contains the inner loop for the optimization and it continues until the tolerance is met.
    """
    x_c = X_c_train[:, 0]
    y_c = X_c_train[:, 1]
    x_b = X_b_train[:, 0]
    y_b = X_b_train[:, 1]
    optimizer.zero_grad()
    loss = loss_function(model, x_c, y_c, x_b, y_b)
    loss.backward()
    global iter
    iter += 1
    print(f" iteration: {iter}  loss: {loss.item()}")
    losses.append(loss.item())
    return loss

def train(model, X_c_train, X_b_train):
    # Initialize the optimizer
    optimizer = torch.optim.LBFGS(model.parameters(),
                                    lr=1,
                                    #max_iter=50000,
                                    #max_eval=50000,
                                    #history_size=50,
                                    tolerance_grad=0,
                                    #tolerance_change=0.5 * np.finfo(float).eps,
                                    line_search_fn="strong_wolfe")

    # the optimizer.step requires the closure function to be a callable function without inputs
    # therefore we need to define a partial function and pass it to the optimizer
    closure_fn = partial(closure, model, optimizer, X_c_train, X_b_train)
    optimizer.step(closure_fn)

In [ ]:
radius = 2 #radius of the circle
theta = 1
sigma = 0.5 #parameters of the OU process

N_c = 250 #number of points in the domain
N_b = 2 * N_c #number of points on the boundary

#definition of X_b_train : N_b points on the boundary
t = np.linspace(0, 2*np.pi, N_b, endpoint=False)
x_b = radius * np.cos(t)
y_b = radius * np.sin(t)
X_b_train = np.vstack( (x_b, y_b) )

#shuffling X_b_train
index = np.arange(0, N_b)
np.random.shuffle(index)
X_b_train = X_b_train[:, index]

#definition of X_c_train : N_c points in the disk
t = np.random.uniform(0,2*np.pi, N_c)
rho = np.random.uniform(0,radius, N_c) ** 0.5 #uniform distribution on the disk
x_c = rho * np.cos(t)
y_c = rho * np.sin(t)
X_c_train = np.vstack( (x_c, y_c) )

#shuffling X_c_train
index = np.arange(0, N_c)
np.random.shuffle(index)
X_c_train = X_c_train[:,index]

In [ ]:
#model instantiation
model = StandardPINN()
model.apply(init_weights)

# Training
losses = []
X_b_train = torch.from_numpy(X_b_train).requires_grad_(True).float()
X_c_train = torch.from_numpy(X_c_train).requires_grad_(True).float()

model.train()
train(model, X_c_train, X_b_train)
plt.plot(losses, label='Training Loss')

In [ ]:
# save the model
torch.save(model.state_dict(), 'standard_pinn_LBFGS.pt')

In [ ]:
#load the model
model = StandardPINN()
model.load_state_dict(\
torch.load('/content/drive/MyDrive/Colab Notebooks/Mean_Escape_Time/models/OU_process/model_standard_pinn_LBFGS.pt'))

#evaluate the model on a uniform grid
n_points = 40
tt = np.linspace(-1, 1, n_points) * radius
xx, yy = np.meshgrid(tt, tt)  # create unit square grid
xx, yy = np.where(xx**2 + yy**2 <= radius**2, xx, 0), np.where(xx**2 + yy**2 <= radius**2 , yy, 0) #(https://stackoverflow.com/questions/15733530/)
zz_true = true_solution_vectorized(xx,yy)

input = torch.from_numpy(np.vstack((xx.ravel(),yy.ravel())).T).float()#.requires_grad_(False)
learned_sol = model(input)

#plot
learned_sol_np = learned_sol.detach().numpy().reshape(xx.shape)
fig, ax = plt.subplots()
ax.plot(cx, cy,'k-', alpha=.2)

contour = ax.contourf(xx, yy, learned_sol_np, levels=500)  # 50 contour levels
cb = fig.colorbar(contour, ax=ax)